# Notebook 01: Addestramento e Valutazione dei Modelli

**Scopo:** Questo notebook carica i dati pre-processati dal Notebook 00, definisce le architetture delle reti neurali, orchestra un ciclo di esperimenti per addestrare e valutare diverse combinazioni di modelli e ottimizzatori, e salva gli artefatti migliori per l'analisi successiva.

**Input:**
- Dati pre-processati da `../data/processed/` (`X_train.npy`, `y_train.npy`, etc.)

**Output (salvati in `../models/` e `../reports/`):**
- I modelli migliori per ogni esperimento (es. `UNet_Lite_Adam.keras`).
- Un file di riepilogo con le metriche di performance (es. `training_summary.csv`).
- (Opzionale) Le storie di training salvate.

In [1]:
# ===================================================================
# CELLA 1: SETUP, IMPORTS E CARICAMENTO DATI
# ===================================================================

import os
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import time
import traceback

import tensorflow as tf
import keras as keras
from keras import layers, models, optimizers, callbacks, regularizers
from keras.utils import to_categorical

# --- Configurazione Globale ---
PROCESSED_DATA_PATH = '../data/processed/'
MODELS_PATH = '../models/ale/'
REPORTS_PATH = '../reports/'
RANDOM_STATE = 42

os.makedirs(MODELS_PATH, exist_ok=True)
os.makedirs(REPORTS_PATH, exist_ok=True)

# 1. GPU e Mixed Precision
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus: tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ GPU(s) Trovata/e: {[tf.config.experimental.get_device_details(g)['device_name'] for g in gpus]}")
        policy = keras.mixed_precision.Policy('mixed_float16')
        keras.mixed_precision.set_global_policy(policy)
        print(f"✅ Politica di Mixed Precision impostata su: {keras.mixed_precision.global_policy().name}")
    except RuntimeError as e: print(f"⚠️ Errore durante l'inizializzazione della GPU: {e}")
else: print("❌ NESSUNA GPU TROVATA. L'allenamento sarà su CPU.")

# 2. Caricamento Dati Pre-processati
print("\n🔄 Caricamento dei dati pre-processati...")
try:
    X_train = np.load(os.path.join(PROCESSED_DATA_PATH, 'X_train.npy'))
    y_train = np.load(os.path.join(PROCESSED_DATA_PATH, 'y_train.npy'))
    X_val = np.load(os.path.join(PROCESSED_DATA_PATH, 'X_val.npy'))
    y_val = np.load(os.path.join(PROCESSED_DATA_PATH, 'y_val.npy'))
    X_test = np.load(os.path.join(PROCESSED_DATA_PATH, 'X_test.npy'))
    y_test = np.load(os.path.join(PROCESSED_DATA_PATH, 'y_test.npy'))
    
    with open(os.path.join(PROCESSED_DATA_PATH, 'label_encoder.pkl'), 'rb') as f:
        label_encoder = pickle.load(f)

    # Conversione in formato categorico
    num_classes = len(label_encoder.classes_)
    y_train_cat = to_categorical(y_train, num_classes=num_classes)
    y_val_cat = to_categorical(y_val, num_classes=num_classes)
    y_test_cat = to_categorical(y_test, num_classes=num_classes)
    
    print("\n✅ Dati caricati con successo.")
    print(f"   - Shape X_train: {X_train.shape} | Shape y_train_cat: {y_train_cat.shape}")
    print(f"   - Numero di classi: {num_classes}")
except FileNotFoundError:
    print("❌ ERRORE: File di dati non trovati. Eseguire prima il notebook '00_Setup_and_Data_Preparation.ipynb'.")

2025-07-25 12:33:54.759810: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-25 12:33:54.770537: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753439634.783584  198303 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753439634.787262  198303 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753439634.797356  198303 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

✅ GPU(s) Trovata/e: ['NVIDIA GeForce RTX 4070']
✅ Politica di Mixed Precision impostata su: mixed_float16

🔄 Caricamento dei dati pre-processati...

✅ Dati caricati con successo.
   - Shape X_train: (2995, 128, 256, 1) | Shape y_train_cat: (2995, 10)
   - Numero di classi: 10


/home/alepot55/Desktop/projects/naml_project/venv/lib/python3.12/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.7.0 when using version 1.7.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [2]:
# ===================================================================
# CELLA 2: DEFINIZIONE DELLE ARCHITETTURE DEI MODELLI (con il nuovo Champion Model)
# ===================================================================
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D, Dropout, BatchNormalization, ReLU,
    Concatenate, GlobalAveragePooling2D, Dense, Reshape, Multiply,
    Activation, SeparableConv2D, add
)

class ModelFactory:
    """
    Contiene le factory per tutte le architetture testate.
    Il nostro nuovo candidato per superare il 73% è 'build_inception_se_cnn'.
    """
    
    @staticmethod
    def _inception_se_block(input_tensor, f1, f3_in, f3_out, f5_in, f5_out, f_pool, se_ratio=16):
        """Helper privato: Blocco Inception con Squeeze-and-Excitation."""
        # Percorso 1: Convoluzione 1x1
        conv1 = Conv2D(filters=f1, kernel_size=(1, 1), padding='same', activation='relu')(input_tensor)

        # Percorso 2: Convoluzione 3x3 con bottleneck
        conv3_bottleneck = Conv2D(filters=f3_in, kernel_size=(1, 1), padding='same', activation='relu')(input_tensor)
        conv3 = Conv2D(filters=f3_out, kernel_size=(3, 3), padding='same', activation='relu')(conv3_bottleneck)

        # Percorso 3: Convoluzione 5x5 con bottleneck
        conv5_bottleneck = Conv2D(filters=f5_in, kernel_size=(1, 1), padding='same', activation='relu')(input_tensor)
        conv5 = Conv2D(filters=f5_out, kernel_size=(5, 5), padding='same', activation='relu')(conv5_bottleneck)

        # Percorso 4: Max-Pooling con bottleneck
        pool = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(input_tensor)
        pool_proj = Conv2D(filters=f_pool, kernel_size=(1, 1), padding='same', activation='relu')(pool)

        # Concatenazione dei percorsi multi-scala
        inception_out = Concatenate(axis=-1)([conv1, conv3, conv5, pool_proj])
        
        # Blocco Squeeze-and-Excitation (SE) per l'attenzione sui canali
        channels = inception_out.shape[-1]
        se = GlobalAveragePooling2D()(inception_out)
        se = Reshape((1, 1, channels))(se)
        se = Dense(channels // se_ratio, activation='relu', use_bias=False)(se)
        se = Dense(channels, activation='sigmoid', use_bias=False)(se)
        
        return Multiply()([inception_out, se])

    @staticmethod
    def build_inception_se_cnn(input_shape, num_classes):
        """
        NUOVO CHAMPION MODEL: Architettura Inception-style con blocchi SE.
        Progettato per efficienza parametrica e cattura di feature multi-scala.
        """
        inputs = Input(shape=input_shape)

        # Blocco Iniziale (Stem)
        x = Conv2D(64, (7, 7), strides=(2, 2), padding='same')(inputs)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

        # Blocco Inception-SE 1
        x = ModelFactory._inception_se_block(x, f1=32, f3_in=48, f3_out=64, f5_in=8, f5_out=16, f_pool=16)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
        x = Dropout(0.3)(x)

        # Blocco Inception-SE 2
        x = ModelFactory._inception_se_block(x, f1=64, f3_in=64, f3_out=128, f5_in=16, f5_out=32, f_pool=32)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
        x = Dropout(0.3)(x)
        
        # Testa di Classificazione
        x = GlobalAveragePooling2D()(x)
        x = Dropout(0.5)(x)
        x = Dense(256, activation='relu')(x)
        x = BatchNormalization()(x)
        outputs = Dense(num_classes, activation='softmax', dtype='float32')(x)

        model = Model(inputs, outputs, name='InceptionSE_CNN')
        return model

    # --- Manteniamo i modelli precedenti per riferimento ---
    @staticmethod
    def build_simple_cnn(input_shape, num_classes):
        """Un solido modello CNN baseline (VGG-style)."""
        # ... (codice invariato)
        inputs = Input(shape=input_shape)
        x = Conv2D(32, (3, 3), padding='same')(inputs)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPooling2D((2, 2))(x)
        x = Conv2D(64, (3, 3), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPooling2D((2, 2))(x)
        x = Conv2D(128, (3, 3), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = GlobalAveragePooling2D()(x)
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.5)(x)
        outputs = Dense(num_classes, activation='softmax', dtype='float32')(x)
        return Model(inputs=inputs, outputs=outputs, name='SimpleCNN')

print("✅ ModelFactory aggiornata. Nuovo candidato 'InceptionSE_CNN' pronto per il test.")

✅ ModelFactory aggiornata. Nuovo candidato 'InceptionSE_CNN' pronto per il test.


In [3]:
# ===================================================================
# CELLA 3: FRAMEWORK DI TRAINING FINALE (Focalizzato sul nuovo modello)
# ===================================================================
import os
import time
import numpy as np
import pandas as pd
import traceback
import tensorflow as tf
from keras import optimizers, callbacks

# -------------------------------------------------------------------
# 1. FUNZIONE DI DATA AUGMENTATION (Invariata)
# -------------------------------------------------------------------
@tf.function
def spec_augment_tf(spectrogram, label):
    """Applica SpecAugment usando solo operazioni TensorFlow."""
    # ... (codice invariato)
    aug_spec = tf.identity(spectrogram)
    freq_bins = tf.shape(aug_spec)[0]
    time_steps = tf.shape(aug_spec)[1]
    
    # Mascheramento in Frequenza
    f_param = tf.cast(tf.cast(freq_bins, tf.float32) * 0.2, tf.int32)
    if f_param > 1:
        f = tf.random.uniform(shape=(), minval=1, maxval=f_param, dtype=tf.int32)
        f0 = tf.random.uniform(shape=(), minval=0, maxval=freq_bins - f, dtype=tf.int32)
        mask_freq_values = tf.concat([tf.ones((f0,), dtype=aug_spec.dtype), tf.zeros((f,), dtype=aug_spec.dtype), tf.ones((freq_bins - f0 - f,), dtype=aug_spec.dtype)], axis=0)
        mask_freq = tf.reshape(mask_freq_values, (freq_bins, 1, 1))
        aug_spec = aug_spec * mask_freq

    # Mascheramento nel Tempo
    t_param = tf.cast(tf.cast(time_steps, tf.float32) * 0.2, tf.int32)
    if t_param > 1:
        t = tf.random.uniform(shape=(), minval=1, maxval=t_param, dtype=tf.int32)
        t0 = tf.random.uniform(shape=(), minval=0, maxval=time_steps - t, dtype=tf.int32)
        mask_time_values = tf.concat([tf.ones((t0,), dtype=aug_spec.dtype), tf.zeros((t,), dtype=aug_spec.dtype), tf.ones((time_steps - t0 - t,), dtype=aug_spec.dtype)], axis=0)
        mask_time = tf.reshape(mask_time_values, (1, time_steps, 1))
        aug_spec = aug_spec * mask_time
        
    return aug_spec, label

# -------------------------------------------------------------------
# 2. CLASSE EVALUATOR (Invariata)
# -------------------------------------------------------------------
class MusicGenreEvaluator:
    """Orchestra il training in modo stabile per tutte le architetture."""
    # ... (codice invariato)
    def __init__(self, class_names):
        self.class_names = class_names
        self.results = []

    def prepare_optimizers(self, lr=1e-3):
        return {
            'Adam': optimizers.Adam(learning_rate=lr),
            'SGD_Momentum': optimizers.SGD(learning_rate=lr*10, momentum=0.9),
            'RMSprop': optimizers.RMSprop(learning_rate=lr)
        }

    def run_experiments(self, model_factories, optimizers_config, train_data, val_data, test_data, epochs):
        for model_name, model_factory in model_factories.items():
            print(f"\n{'='*80}\nARCHITETTURA IN TEST: '{model_name}'\n{'='*80}")
            for optimizer_name, optimizer in optimizers_config.items():
                print(f"\n--- 🚀 TRAINING: [{model_name}] with [{optimizer_name}] ---")
                try:
                    model = model_factory()
                    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
                    
                    callbacks_list = [
                        callbacks.EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True, verbose=1),
                        callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=1),
                        callbacks.ModelCheckpoint(os.path.join(MODELS_PATH, f"{model_name}_{optimizer_name}.keras"),
                                                  monitor='val_accuracy', save_best_only=True)
                    ]
                    
                    history = model.fit(train_data, epochs=epochs, validation_data=val_data, callbacks=callbacks_list, verbose=2)
                    
                    test_loss, test_acc = model.evaluate(test_data, verbose=0)
                    self.results.append({
                        'Experiment': f"{model_name}_{optimizer_name}", 'Model': model_name, 'Optimizer': optimizer_name,
                        'Test_Accuracy': test_acc, 'Test_Loss': test_loss,
                        'Best_Val_Accuracy': max(history.history['val_accuracy']),
                        'Epochs': len(history.history['val_accuracy']),
                    })
                except Exception:
                    print(f"❌ ERRORE durante il training di [{model_name}] con [{optimizer_name}]:")
                    traceback.print_exc()
        return pd.DataFrame(self.results)

# -------------------------------------------------------------------
# 3. ESECUZIONE DEL CICLO DI TRAINING SUL NUOVO MODELLO
# -------------------------------------------------------------------
AUTOTUNE = tf.data.AUTOTUNE
BATCH_SIZE = 64 # Manteniamo un batch size ragionevole per Inception
EPOCHS = 100

# Impostazioni globali (invariate)
tf.keras.mixed_precision.set_global_policy('float32')

# Pipeline di dati (invariata)
train_pipeline = (tf.data.Dataset.from_tensor_slices((X_train, y_train_cat)).cache().shuffle(len(X_train))
                  .map(spec_augment_tf, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE))
val_pipeline = (tf.data.Dataset.from_tensor_slices((X_val, y_val_cat)).cache().batch(BATCH_SIZE).prefetch(AUTOTUNE))
test_pipeline = (tf.data.Dataset.from_tensor_slices((X_test, y_test_cat)).cache().batch(BATCH_SIZE).prefetch(AUTOTUNE))

# *** MODIFICA CHIAVE ***
# Factory dei modelli: puntiamo solo al nostro nuovo candidato.
input_shape = X_train.shape[1:]
num_classes = y_train_cat.shape[1]
model_factories = {
    'InceptionSE_CNN': lambda: ModelFactory.build_inception_se_cnn(input_shape, num_classes),
}

# Esecuzione
evaluator = MusicGenreEvaluator(class_names=label_encoder.classes_)
optimizers_config = evaluator.prepare_optimizers(lr=1e-3) # Adam funziona bene con 1e-3
results_df = evaluator.run_experiments(
    model_factories, optimizers_config, train_pipeline, val_pipeline, test_pipeline, EPOCHS
)

# Salvataggio e visualizzazione (invariati)
if not results_df.empty:
    results_df.to_csv(os.path.join(REPORTS_PATH, 'training_summary_InceptionSE.csv'), index=False)
    print("\n🎉 CICLO DI TRAINING SUL NUOVO MODELLO COMPLETATO 🎉")
    print("\nRisultati Finali:")
    print(results_df.sort_values(by='Best_Val_Accuracy', ascending=False))

I0000 00:00:1753439636.303748  198303 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10162 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070, pci bus id: 0000:01:00.0, compute capability: 8.9



ARCHITETTURA IN TEST: 'InceptionSE_CNN'

--- 🚀 TRAINING: [InceptionSE_CNN] with [Adam] ---
Epoch 1/100


I0000 00:00:1753439640.723895  199115 service.cc:152] XLA service 0x7c40bc00d160 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753439640.723923  199115 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4070, Compute Capability 8.9
2025-07-25 12:34:00.816660: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1753439641.511187  199115 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753439652.377420  199115 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2025-07-25 12:34:31.988426: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_385', 28 bytes spill stores, 28 bytes spill loads



47/47 - 36s - 776ms/step - accuracy: 0.3162 - loss: 2.0669 - val_accuracy: 0.1000 - val_loss: 2.3745 - learning_rate: 1.0000e-03
Epoch 2/100
47/47 - 1s - 29ms/step - accuracy: 0.4311 - loss: 1.5994 - val_accuracy: 0.1520 - val_loss: 2.4946 - learning_rate: 1.0000e-03
Epoch 3/100
47/47 - 1s - 30ms/step - accuracy: 0.5018 - loss: 1.4176 - val_accuracy: 0.1890 - val_loss: 2.5236 - learning_rate: 1.0000e-03
Epoch 4/100
47/47 - 1s - 28ms/step - accuracy: 0.5309 - loss: 1.3042 - val_accuracy: 0.1540 - val_loss: 2.5494 - learning_rate: 1.0000e-03
Epoch 5/100
47/47 - 1s - 30ms/step - accuracy: 0.5780 - loss: 1.1880 - val_accuracy: 0.2260 - val_loss: 2.2622 - learning_rate: 1.0000e-03
Epoch 6/100
47/47 - 1s - 27ms/step - accuracy: 0.6140 - loss: 1.0938 - val_accuracy: 0.1570 - val_loss: 3.1061 - learning_rate: 1.0000e-03
Epoch 7/100
47/47 - 1s - 29ms/step - accuracy: 0.6361 - loss: 1.0345 - val_accuracy: 0.2540 - val_loss: 2.1581 - learning_rate: 1.0000e-03
Epoch 8/100
47/47 - 1s - 30ms/step - 

2025-07-25 12:35:37.611333: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_3095', 388 bytes spill stores, 388 bytes spill loads

2025-07-25 12:35:37.928980: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_3095', 416 bytes spill stores, 416 bytes spill loads

2025-07-25 12:35:37.963038: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_3095', 400 bytes spill stores, 400 bytes spill loads

2025-07-25 12:35:37.995794: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_3095', 680 bytes spill stores, 548 bytes spill loads

2025-07-25 12:35:46.377119: I ex

47/47 - 23s - 480ms/step - accuracy: 0.2531 - loss: 2.2507 - val_accuracy: 0.1440 - val_loss: 2.2291 - learning_rate: 0.0100
Epoch 2/100
47/47 - 1s - 29ms/step - accuracy: 0.4050 - loss: 1.6236 - val_accuracy: 0.1640 - val_loss: 2.3140 - learning_rate: 0.0100
Epoch 3/100
47/47 - 1s - 28ms/step - accuracy: 0.4654 - loss: 1.4793 - val_accuracy: 0.1470 - val_loss: 2.3294 - learning_rate: 0.0100
Epoch 4/100
47/47 - 1s - 28ms/step - accuracy: 0.4898 - loss: 1.3745 - val_accuracy: 0.1360 - val_loss: 2.3437 - learning_rate: 0.0100
Epoch 5/100
47/47 - 1s - 28ms/step - accuracy: 0.5329 - loss: 1.2913 - val_accuracy: 0.1490 - val_loss: 2.5755 - learning_rate: 0.0100
Epoch 6/100

Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.
47/47 - 1s - 31ms/step - accuracy: 0.5359 - loss: 1.2690 - val_accuracy: 0.2150 - val_loss: 2.4088 - learning_rate: 0.0100
Epoch 7/100
47/47 - 1s - 31ms/step - accuracy: 0.5716 - loss: 1.1783 - val_accuracy: 0.2730 - val_loss: 2.2273 - learning_r